In [31]:
import os
import argparse
import numpy as np
from PIL import Image
import open3d as o3d
from tqdm import tqdm
import torch
import torch.nn.functional as F
from torchvision import transforms
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
import json

## Models

In [32]:
class DINOv2Dense:
    def __init__(self):
        self.model = torch.hub.load("facebookresearch/dinov2", "dinov2_vits14")
        self.model.eval()

        # Standard DINOv2 transforms
        self.preprocess = transforms.Compose([
            transforms.Resize(518, interpolation=transforms.InterpolationMode.BICUBIC),
            transforms.CenterCrop(518),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]
            ),
        ])

    def get_dense_features(self, image):

        # Convert RGBA → RGB
        if not isinstance(image, torch.Tensor):
            if image.mode != "RGB":
                image = image.convert("RGB")

            image = self.preprocess(image).unsqueeze(0)

        with torch.no_grad():
            feats = self.model.forward_features(image)

        patch_tokens = feats["x_norm_patchtokens"]  # [B, N, C]

        B, N, C = patch_tokens.shape
        H = W = int(N ** 0.5)

        patch_tokens = patch_tokens.view(B, H, W, C).permute(0, 3, 1, 2)
        patch_tokens = F.normalize(patch_tokens, dim=1)

        return patch_tokens

## PIPELINE

### 1. OUTLIER REMOVAL

In [33]:
def remove_outliers(filepath):
    pcd = o3d.io.read_point_cloud(filepath)
    cl, ind = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
    inlier_cloud = pcd.select_by_index(ind)
    o3d.io.write_point_cloud(filepath, inlier_cloud)

### 2. AFFORDANCE DETECTION

#### IO HELPERS

In [34]:
def read_cameras_txt(path):
    cams = {}
    with open(path, 'r') as f:
        for line in f:
            if not line.strip() or line.startswith('#'): continue
            p = line.split()
            cid = int(p[0]); model = p[1]; w = int(p[2]); h = int(p[3])
            params = np.array([float(x) for x in p[4:]], dtype=np.float64)
            cams[cid] = {'model': model, 'w': w, 'h': h, 'params': params}
    return cams

def read_images_txt(path):
    imgs = {}
    with open(path, 'r') as f:
        lines = [l.strip() for l in f if l.strip() and not l.startswith('#')]
    i = 0
    while i < len(lines):
        parts = lines[i].split()
        iid = int(parts[0])
        qvec = np.array(list(map(float, parts[1:5])), dtype=np.float64)
        tvec = np.array(list(map(float, parts[5:8])), dtype=np.float64)
        cam_id = int(parts[8]); name = parts[9]
        imgs[iid] = {'qvec': qvec, 'tvec': tvec, 'cam_id': cam_id, 'name': name}
        i += 2
    return imgs

def read_points3D_txt(path):
    pts = []
    cols = []
    with open(path, "r") as f:
        for line in f:
            if line.startswith("#") or len(line.strip()) < 2:
                continue
            vals = line.split()
            # FORMAT: ID, X, Y, Z, R, G, B, ERROR, (IMAGE-ID POINT-2D-ID)...
            X = float(vals[1])
            Y = float(vals[2])
            Z = float(vals[3])
            R = int(vals[4]) / 255.0
            G = int(vals[5]) / 255.0
            B = int(vals[6]) / 255.0
            pts.append([X, Y, Z])
            cols.append([R, G, B])

    return np.array(pts, dtype=np.float32), np.array(cols, dtype=np.float32)

#### MATH TO TRAUMATISE

In [35]:
def qvec2rotmat(q):
    # Quaternion -> Rotation matrix
    qw, qx, qy, qz = q
    return np.array([
        [1-2*(qy*qy+qz*qz), 2*(qx*qy-qz*qw),   2*(qx*qz+qy*qw)],
        [2*(qx*qy+qz*qw),   1-2*(qx*qx+qz*qz), 2*(qy*qz-qx*qw)],
        [2*(qx*qz-qy*qw),   2*(qy*qz+qx*qw),   1-2*(qx*qx+qy*qy)]
    ], dtype=np.float64)


def project_points(pts, R, t, params):
    # Pinhole projection: u = fx * Xc/Zc + cx ; v = fy * Yc/Zc + cy
    Xc = (R @ pts.T).T + t.reshape(1,3)         # (N,3)
    Z = Xc[:,2]
    eps = 1e-9
    x = Xc[:,0] / (Z + eps); y = Xc[:,1] / (Z + eps)
    fx, fy, cx, cy = params[:4]
    u = fx * x + cx; v = fy * y + cy
    return u, v, Z

def bilinear_sample(feat, us, vs):
    # Bilinear interpolation of HxWxC feature map
    H, W, C = feat.shape
    us_f = np.asarray(us); vs_f = np.asarray(vs)
    i = np.floor(us_f).astype(int); j = np.floor(vs_f).astype(int)
    i0 = np.clip(i, 0, W-1); j0 = np.clip(j, 0, H-1)
    i1 = np.clip(i0+1, 0, W-1); j1 = np.clip(j0+1, 0, H-1)
    a = (us_f - i)[:,None]; b = (vs_f - j)[:,None]
    feat_np = np.asarray(feat)
    f00 = feat_np[j0, i0]; f01 = feat_np[j0, i1]; f10 = feat_np[j1, i0]; f11 = feat_np[j1, i1]
    return (1-a)*(1-b)*f00 + a*(1-b)*f01 + (1-a)*b*f10 + a*b*f11

#### FEATURE AND TEXT ENCODERS

In [36]:
class ImageFeatureExtractor:
    def __init__(self, dino_model):
        self.dino = dino_model        # instance of DINOv2Dense

    def load_embedding(self, img_path):
        img = Image.open(img_path)
        emb_up = self.dino.get_dense_features(img)
        # emb_up: [1, C, H, W]
        return emb_up.squeeze(0)      # → [C, H, W]

In [37]:
class PointProjector:
    def __init__(self, pts):
        self.pts = pts                # (N, 3)

    def project(self, pts, R, t, cam_params):
        return project_points(pts, R, t, cam_params)

    def zbuffer_filter(self, u, v, z, H, W, z_eps):
        idxs = np.where((z > 0) & (u >= 0) & (u < W - 1) & (v >= 0) & (v < H - 1))[0]
        if len(idxs) == 0:
            return None

        us = np.floor(u[idxs]).astype(int)
        vs = np.floor(v[idxs]).astype(int)
        flat = vs * W + us

        zbuf_flat = np.full(H * W, np.inf, dtype=np.float32)
        np.minimum.at(zbuf_flat, flat, z[idxs].astype(np.float32))
        zbuf = zbuf_flat.reshape(H, W)

        z_at_pix = zbuf[vs, us]
        tol = np.maximum(z_eps, 0.01 * z[idxs])

        keep = idxs[z[idxs] <= (z_at_pix + tol)]
        return keep


In [38]:
class FeatureFusion:
    def __init__(self, N, C):
        self.sum_feats = np.zeros((N, C), dtype=np.float32)
        self.counts = np.zeros(N, dtype=np.float32)

    def accumulate(self, idxs, sampled, weights):
        self.sum_feats[idxs] += sampled * weights[:, None]
        self.counts[idxs] += weights

    def finalize(self):
        counts = np.maximum(self.counts, 1e-8)[:, None]
        feats = self.sum_feats / counts
        feats /= (np.linalg.norm(feats, axis=1, keepdims=True) + 1e-9)
        return feats       # (N, C normalized)


In [39]:

class ClusteringModule:
    def __init__(self, eps=0.5, min_samples=10):
        self.eps = eps
        self.min_samples = min_samples

    def cluster(self, pts, feats):
        # Concatenate 3D pos + C-dim embedding
        data = np.concatenate([pts, feats], axis=1)    # (N, 3+C)
        labels = DBSCAN(eps=self.eps, min_samples=self.min_samples).fit_predict(data)
        return labels


In [40]:
def run_seg_pipeline(colmap_txt_dir, images_dir, out_ply,
                 down=1, max_images=None, z_eps=0.01):

    # ---------------------------------------------
    # Load point cloud + COLMAP metadata
    # ---------------------------------------------
    pts_path = os.path.join(colmap_txt_dir, "points3D.txt")
    pts, colors = read_points3D_txt(pts_path)   # shape (N,3)
    cams = read_cameras_txt(os.path.join(colmap_txt_dir,'cameras.txt'))
    imgs = read_images_txt(os.path.join(colmap_txt_dir,'images.txt'))
    img_items = list(imgs.items())[:max_images] if max_images else list(imgs.items())

    # ---------------------------------------------
    # Initialize modules
    # ---------------------------------------------
    dino_model = DINOv2Dense()                     # your existing file
    extractor   = ImageFeatureExtractor(dino_model)
    projector   = PointProjector(pts)

    # We must know DINO embedding size → get from the first image
    first_img = Image.open(os.path.join(images_dir, img_items[0][1]['name']))
    emb_up = dino_model.get_dense_features(first_img)
    C = emb_up.shape[1]

    fusion = FeatureFusion(len(pts), C)

    # ---------------------------------------------
    # Loop over images
    # ---------------------------------------------
    for _, info in tqdm(img_items, desc="Images"):

        img_path = os.path.join(images_dir, info['name'])
        if not os.path.exists(img_path): 
            continue

        R = qvec2rotmat(info['qvec'])
        t = info['tvec']
        cam = cams[info['cam_id']]

        # DINO embedding
        emb = extractor.load_embedding(img_path)   # [C, H, W]
        C_emb, H, W = emb.shape

        # Project points
        u, v, z = projector.project(pts, R, t, cam['params'])
        if down > 1:
            u, v = u / down, v / down

        # Z-buffer
        keep = projector.zbuffer_filter(u, v, z, H, W, z_eps)
        if keep is None or len(keep) == 0:
            continue

        us = u[keep]
        vs = v[keep]

        # Sample DINO pixels → C-dim embeddings
        sampled = bilinear_sample(emb.permute((1, 2, 0)), us, vs)   # (M, C)

        # Weight by view angle + distance
        cam_center = -R.T.dot(t)
        vec = cam_center - pts[keep]
        d = np.linalg.norm(vec, axis=1) + 1e-9
        view = vec / d[:,None]
        forward = R.T.dot(np.array([0,0,1],dtype=np.float64))
        cos = np.clip(view.dot(forward), 0, 1)
        w = cos / (d**2 + 1e-6)

        fusion.accumulate(keep, sampled, w)

    # ---------------------------------------------
    # Fused per-point DINO embedding
    # ---------------------------------------------
    feats = fusion.finalize()

    # ---------------------------------------------
    # Cluster features + xyz
    # ---------------------------------------------
    clusterer = ClusteringModule(eps=0.35, min_samples=15)
    labels = clusterer.cluster(pts, feats)          # (N,)

    # label colors (handle noise label -1)
    max_label = labels.max()
    label_colors = np.random.rand(max_label + 1, 3)
    point_colors = np.zeros((len(labels), 3))

    for i, lab in enumerate(labels):
        if lab == -1:
            point_colors[i] = [0, 0, 0]      # noise = black
        else:
            point_colors[i] = label_colors[lab]

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pts)
    pcd.colors = o3d.utility.Vector3dVector(point_colors)
    o3d.io.write_point_cloud(out_ply, pcd)

    print("Saved clustered:", out_ply)

    return pts, labels, feats, colors

In [ ]:
class MeshGenerator:
    def __init__(self, out_dir="output"):
        self.out_dir = out_dir
        os.makedirs(out_dir, exist_ok=True)
        self.metadata = {}   # store cluster metadata

    # -----------------------------------------------------
    # Normal estimation
    # -----------------------------------------------------
    def draw_normals(self, pcd):
        pcd = pcd.voxel_down_sample(0.005)
        pcd.estimate_normals(
            search_param=o3d.geometry.KDTreeSearchParamHybrid(
                radius=0.01, max_nn=30
            )
        )

        pcd = pcd.uniform_down_sample(every_k_points=2) # special code 
        pcd.orient_normals_consistent_tangent_plane(30)
        return pcd

    # -----------------------------------------------------
    # Poisson reconstruction + density filtering
    # -----------------------------------------------------
    def create_mesh(self, pcd):
        if len(pcd.points) > 1000:
            pcd, _ = pcd.remove_statistical_outlier(30, 2.0)

        pcd = pcd.voxel_down_sample(0.003)

        if len(pcd.points) < 10:
            print("❌ Too few points for meshing.")
            return None

        print("Running Poisson...")

        try:
            mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
                pcd, depth=10
            )
        except:
            print("❌ Poisson crashed.")
            return None

        if len(mesh.vertices) == 0:
            print("❌ Empty Poisson mesh.")
            return None

        densities = np.asarray(densities)
        keep_mask = densities > np.quantile(densities, 0.05)

        verts = np.asarray(mesh.vertices)
        tris = np.asarray(mesh.triangles)

        keep_idx = np.where(keep_mask)[0]
        if len(keep_idx) < 20:
            print("❌ Filtering removed all vertices.")
            return None

        remap = -np.ones(len(verts), dtype=int)
        remap[keep_idx] = np.arange(len(keep_idx))

        good_tris = tris[np.all(keep_mask[tris], axis=1)]
        good_tris = remap[good_tris]

        new_mesh = o3d.geometry.TriangleMesh(
            o3d.utility.Vector3dVector(verts[keep_idx]),
            o3d.utility.Vector3iVector(good_tris.astype(np.int32))
        )

        new_mesh.remove_unreferenced_vertices()
        new_mesh.compute_vertex_normals()

        return new_mesh

    # -----------------------------------------------------
    # Nearest-neighbor color transfer from original PCD
    # -----------------------------------------------------
    def color_mesh_from_pcd(self, mesh, original_cluster_pcd):
        if not original_cluster_pcd.has_colors():
            return mesh

        kdtree = o3d.geometry.KDTreeFlann(original_cluster_pcd)
        colors = []

        for v in mesh.vertices:
            _, idx, _ = kdtree.search_knn_vector_3d(v, 1)
            colors.append(original_cluster_pcd.colors[idx[0]])

        mesh.vertex_colors = o3d.utility.Vector3dVector(np.array(colors))
        return mesh

    # -----------------------------------------------------
    # Save all cluster metadata
    # -----------------------------------------------------
    def save_metadata(self):
        clean_meta = self._to_python_type(self.metadata)

        with open(os.path.join(self.out_dir, "meta.json"), "w") as f:
            json.dump(clean_meta, f, indent=4)
        print("✓ meta.json saved.")

    # -----------------------------------------------------
    # Process a single cluster
    # -----------------------------------------------------
    def process_cluster(self, cluster_id, all_points, all_colors, labels, feats):
        idx = np.where(labels == cluster_id)[0]

        if len(idx) < 30:
            print(f"Cluster {cluster_id} too small → skip")
            return

        pts_c = all_points[idx]
        col_c = all_colors[idx] if all_colors is not None else None
        feat_c = feats[idx]

        # Build Open3D PCD
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(pts_c)
        if col_c is not None:
            pcd.colors = o3d.utility.Vector3dVector(col_c)

        # Compute centroid and feature mean
        centroid = pts_c.mean(axis=0).tolist()
        feat_mean = feat_c.mean(axis=0).tolist()

        # Create folder output/meshX/
        cluster_dir = os.path.join(self.out_dir, f"mesh{cluster_id}")
        os.makedirs(cluster_dir, exist_ok=True)

        # Save feature info
        np.save(
            os.path.join(cluster_dir, "features.npy"),
            {"point_ids": idx, "feature_mean": feat_mean}
        )

        # Metadata entry
        self.metadata[str(cluster_id)] = {
            "cluster_id": cluster_id,
            "centroid": centroid,
            "num_points": len(idx),
            "feature_mean": feat_mean,
            "point_ids": idx.tolist(),
            "visual_mesh": f"mesh{cluster_id}/visual.ply",
            "collision_mesh": f"mesh{cluster_id}/collision.ply"
        }

        # Normal estimation
        pcd_norm = self.draw_normals(pcd)

        # Meshing
        mesh = self.create_mesh(pcd_norm)
        if mesh is None:
            print(f"Cluster {cluster_id} → meshing failed")
            return

        mesh = self.color_mesh_from_pcd(mesh, pcd)

        # Save visual mesh
        vis_path = os.path.join(cluster_dir, "visual.ply")
        o3d.io.write_triangle_mesh(vis_path, mesh)

        # Collision mesh
        collision = mesh.simplify_quadric_decimation(1500)
        col_path = os.path.join(cluster_dir, "collision.ply")
        o3d.io.write_triangle_mesh(col_path, collision)

        print(f"✓ Mesh saved for cluster {cluster_id}")

    # -----------------------------------------------------
    # Run for ALL clusters
    # -----------------------------------------------------
    def run(self, all_points, all_colors, labels, feats):
        clusters = np.unique(labels)
        for c in clusters:
            print(f"\n=== Cluster {c} ===")
            self.process_cluster(c, all_points, all_colors, labels, feats)

        self.save_metadata()
    
    def _to_python_type(self, obj):
        """Recursively convert numpy types to native Python types"""
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        if isinstance(obj, list):
            return [self._to_python_type(i) for i in obj]
        if isinstance(obj, dict):
            return {k: self._to_python_type(v) for k, v in obj.items()}
        return obj


In [42]:
colmap_txt_dir = "../frames/sparse/0"
images_dir     = "../frames/images"
out_ply        = "clustered_output.ply"


pts, labels, feats, color = run_seg_pipeline(colmap_txt_dir, images_dir, out_ply, down = 2)

Using cache found in /home/raman/.cache/torch/hub/facebookresearch_dinov2_main
Images: 100%|██████████| 469/469 [04:47<00:00,  1.63it/s]


Saved clustered: clustered_output.ply


### Divide and generate mesh

In [43]:
mesh_gen = MeshGenerator("output")
mesh_gen.run(pts, color, labels, feats)


=== Cluster 0 ===
Running Poisson...
✓ Mesh saved for cluster 0

=== Cluster 1 ===
Running Poisson...
✓ Mesh saved for cluster 1

=== Cluster 2 ===
Running Poisson...
✓ Mesh saved for cluster 2

=== Cluster 3 ===
Running Poisson...
✓ Mesh saved for cluster 3

=== Cluster 4 ===
Cluster 4 too small → skip

=== Cluster 5 ===
Running Poisson...
✓ Mesh saved for cluster 5

=== Cluster 6 ===
Running Poisson...
✓ Mesh saved for cluster 6

=== Cluster 7 ===
Running Poisson...
✓ Mesh saved for cluster 7

=== Cluster 8 ===
Cluster 8 too small → skip

=== Cluster 9 ===
Running Poisson...
✓ Mesh saved for cluster 9

=== Cluster 10 ===
Running Poisson...
✓ Mesh saved for cluster 10

=== Cluster 11 ===
Running Poisson...
✓ Mesh saved for cluster 11

=== Cluster 12 ===
Running Poisson...
✓ Mesh saved for cluster 12

=== Cluster 13 ===
Running Poisson...
✓ Mesh saved for cluster 13

=== Cluster 14 ===
Running Poisson...
✓ Mesh saved for cluster 14

=== Cluster 15 ===
Running Poisson...
✓ Mesh saved f

In [46]:
import os
import json
import numpy as np
import subprocess


class GazeboModelBuilder:
    def __init__(self, output_root="output", gazebo_root="gazebo_model"):
        self.output_root = output_root
        self.gazebo_root = gazebo_root

        self.mesh_dir = os.path.join(gazebo_root, "meshes")
        os.makedirs(self.mesh_dir, exist_ok=True)

        # store only successful clusters
        self.valid_clusters = {}

    # --------------------------------------------------------------
    # Convert PLY → DAE, return True/False
    # --------------------------------------------------------------
    def convert_ply_to_dae(self, ply_path, dae_path):
        if not os.path.exists(ply_path):
            print(f"❌ Missing file: {ply_path}")
            return False

        cmd = [
            "meshlabserver",
            "-i", ply_path,
            "-o", dae_path,
            "-m", "vc"
        ]

        try:
            subprocess.run(cmd, check=True)
            if os.path.exists(dae_path):
                print(f"✓ Saved DAE: {dae_path}")
                return True
            else:
                print(f"❌ DAE not created: {dae_path}")
                return False

        except Exception as e:
            print(f"❌ Error: {e}")
            return False

    # --------------------------------------------------------------
    def write_model_config(self):
        config_path = os.path.join(self.gazebo_root, "model.config")
        with open(config_path, "w") as f:
            f.write("""<?xml version="1.0"?>
<model>
  <name>unified_scene</name>
  <version>1.0</version>
  <sdf version="1.6">model.sdf</sdf>
</model>""")
        print(f"✓ Wrote {config_path}")

    # --------------------------------------------------------------
    # Write SDF only for valid clusters
    # --------------------------------------------------------------
    def write_model_sdf(self):
        sdf_path = os.path.join(self.gazebo_root, "model.sdf")

        with open(sdf_path, "w") as f:
            f.write("""<?xml version="1.0"?>
<sdf version="1.6">
  <model name="unified_scene">
    <static>true</static>
""")

        with open(sdf_path, "a") as f:
            for cluster_id, entry in self.valid_clusters.items():

                centroid = entry["centroid"]
                vis_dae = f"meshes/mesh{cluster_id}_visual.dae"
                col_dae = f"meshes/mesh{cluster_id}_collision.dae"

                f.write(f"""
    <link name="cluster_{cluster_id}">
      <pose>{centroid[0]} {centroid[1]} {centroid[2]} 0 0 0</pose>

      <visual name="visual_{cluster_id}">
        <geometry>
          <mesh>
            <uri>model://unified_scene/{vis_dae}</uri>
            <scale>1 1 1</scale>
          </mesh>
        </geometry>
      </visual>

      <collision name="collision_{cluster_id}">
        <geometry>
          <mesh>
            <uri>model://unified_scene/{col_dae}</uri>
            <scale>1 1 1</scale>
          </mesh>
        </geometry>
      </collision>
    </link>
""")

        with open(sdf_path, "a") as f:
            f.write("""
  </model>
</sdf>
""")

        print(f"✓ Wrote {sdf_path}")

    # --------------------------------------------------------------
    def build(self):
        meta_path = os.path.join(self.output_root, "meta.json")

        if not os.path.exists(meta_path):
            print("❌ meta.json missing")
            return

        meta = json.load(open(meta_path))
        meta = self._convert_meta(meta)

        print("✓ Loaded metadata")

        # Convert only existing meshes
        for cluster_id, entry in meta.items():

            mesh_dir = os.path.join(self.output_root, f"mesh{cluster_id}")
            visual_ply = os.path.join(mesh_dir, "visual.ply")
            collision_ply = os.path.join(mesh_dir, "collision.ply")

            visual_dae = os.path.join(self.mesh_dir, f"mesh{cluster_id}_visual.dae")
            collision_dae = os.path.join(self.mesh_dir, f"mesh{cluster_id}_collision.dae")

            ok_visual = self.convert_ply_to_dae(visual_ply, visual_dae)
            ok_collision = self.convert_ply_to_dae(collision_ply, collision_dae)

            if ok_visual and ok_collision:
                self.valid_clusters[cluster_id] = entry
            else:
                print(f"⚠️ Skipping cluster {cluster_id} due to failure.")

        self.write_model_config()
        self.write_model_sdf()

        print("\n🎉 DONE — Meshes successfully included:", len(self.valid_clusters))
        print("   Your Gazebo model will NOT crash now.")

    # --------------------------------------------------------------
    def _convert_meta(self, obj):
        if isinstance(obj, dict):
            return {k: self._convert_meta(v) for k, v in obj.items()}
        if isinstance(obj, list):
            return [self._convert_meta(x) for x in obj]
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        if isinstance(obj, (np.integer,)):
            return int(obj)
        if isinstance(obj, (np.floating,)):
            return float(obj)
        return obj


In [47]:
builder = GazeboModelBuilder(output_root="output", gazebo_root="gazebo_model")
builder.build()

✓ Loaded metadata


Current Plugins Dir is: /usr/lib/x86_64-linux-gnu/meshlab/plugins 
Opening a file with extension ply


Loading Plugins:
Total 266 filtering actions
Total 13 io plugins
Mesh /home/raman/Desktop/Projects/KineticsVerse/noteboooks/output/mesh0/visual.ply loaded has 111904 vn 219278 fn
output mesh  /home/raman/Desktop/Projects/KineticsVerse/noteboooks/gazebo_model/meshes/mesh0_visual.dae
vertex color, Mesh /home/raman/Desktop/Projects/KineticsVerse/noteboooks/output/mesh0/visual.ply saved as /home/raman/Desktop/Projects/KineticsVerse/noteboooks/gazebo_model/meshes/mesh0_visual.dae (111904 vn 219278 fn)
✓ Saved DAE: gazebo_model/meshes/mesh0_visual.dae
Loading Plugins:
Total 266 filtering actions
Total 13 io plugins
Mesh /home/raman/Desktop/Projects/KineticsVerse/noteboooks/output/mesh0/collision.ply loaded has 3023 vn 2250 fn
output mesh  /home/raman/Desktop/Projects/KineticsVerse/noteboooks/gazebo_model/meshes/mesh0_collision.dae
vertex color, Mesh /home/raman/Desktop/Projects/KineticsVerse/noteboooks/output/mesh0/collision.ply saved as /home/raman/Desktop/Projects/KineticsVerse/noteboooks/

Current Plugins Dir is: /usr/lib/x86_64-linux-gnu/meshlab/plugins 
Opening a file with extension ply
Current Plugins Dir is: /usr/lib/x86_64-linux-gnu/meshlab/plugins 
Opening a file with extension ply
Current Plugins Dir is: /usr/lib/x86_64-linux-gnu/meshlab/plugins 
Opening a file with extension ply


Loading Plugins:
Total 266 filtering actions
Total 13 io plugins
Mesh /home/raman/Desktop/Projects/KineticsVerse/noteboooks/output/mesh1/visual.ply loaded has 2008 vn 3865 fn
output mesh  /home/raman/Desktop/Projects/KineticsVerse/noteboooks/gazebo_model/meshes/mesh1_visual.dae
vertex color, Mesh /home/raman/Desktop/Projects/KineticsVerse/noteboooks/output/mesh1/visual.ply saved as /home/raman/Desktop/Projects/KineticsVerse/noteboooks/gazebo_model/meshes/mesh1_visual.dae (2008 vn 3865 fn)
✓ Saved DAE: gazebo_model/meshes/mesh1_visual.dae
Loading Plugins:
Total 266 filtering actions
Total 13 io plugins
Mesh /home/raman/Desktop/Projects/KineticsVerse/noteboooks/output/mesh1/collision.ply loaded has 786 vn 1499 fn
output mesh  /home/raman/Desktop/Projects/KineticsVerse/noteboooks/gazebo_model/meshes/mesh1_collision.dae
vertex color, Mesh /home/raman/Desktop/Projects/KineticsVerse/noteboooks/output/mesh1/collision.ply saved as /home/raman/Desktop/Projects/KineticsVerse/noteboooks/gazebo_mo

...

Loading Plugins:
Total 266 filtering actions
Total 13 io plugins
Mesh /home/raman/Desktop/Projects/KineticsVerse/noteboooks/output/mesh25/visual.ply loaded has 1277 vn 2387 fn
output mesh  /home/raman/Desktop/Projects/KineticsVerse/noteboooks/gazebo_model/meshes/mesh25_visual.dae
vertex color, Mesh /home/raman/Desktop/Projects/KineticsVerse/noteboooks/output/mesh25/visual.ply saved as /home/raman/Desktop/Projects/KineticsVerse/noteboooks/gazebo_model/meshes/mesh25_visual.dae (1277 vn 2387 fn)
✓ Saved DAE: gazebo_model/meshes/mesh25_visual.dae
Loading Plugins:
Total 266 filtering actions
Total 13 io plugins
Mesh /home/raman/Desktop/Projects/KineticsVerse/noteboooks/output/mesh25/collision.ply loaded has 813 vn 1500 fn
output mesh  /home/raman/Desktop/Projects/KineticsVerse/noteboooks/gazebo_model/meshes/mesh25_collision.dae
vertex color, Mesh /home/raman/Desktop/Projects/KineticsVerse/noteboooks/output/mesh25/collision.ply saved as /home/raman/Desktop/Projects/KineticsVerse/noteboooks/g

Opening a file with extension ply
Current Plugins Dir is: /usr/lib/x86_64-linux-gnu/meshlab/plugins 
Opening a file with extension ply


Loading Plugins:
Total 266 filtering actions
Total 13 io plugins
Mesh /home/raman/Desktop/Projects/KineticsVerse/noteboooks/output/mesh29/visual.ply loaded has 1507 vn 2815 fn
output mesh  /home/raman/Desktop/Projects/KineticsVerse/noteboooks/gazebo_model/meshes/mesh29_visual.dae
vertex color, Mesh /home/raman/Desktop/Projects/KineticsVerse/noteboooks/output/mesh29/visual.ply saved as /home/raman/Desktop/Projects/KineticsVerse/noteboooks/gazebo_model/meshes/mesh29_visual.dae (1507 vn 2815 fn)
✓ Saved DAE: gazebo_model/meshes/mesh29_visual.dae
Loading Plugins:
Total 266 filtering actions
Total 13 io plugins
Mesh /home/raman/Desktop/Projects/KineticsVerse/noteboooks/output/mesh29/collision.ply loaded has 818 vn 1500 fn
output mesh  /home/raman/Desktop/Projects/KineticsVerse/noteboooks/gazebo_model/meshes/mesh29_collision.dae
vertex color, Mesh /home/raman/Desktop/Projects/KineticsVerse/noteboooks/output/mesh29/collision.ply saved as /home/raman/Desktop/Projects/KineticsVerse/noteboooks/g

Current Plugins Dir is: /usr/lib/x86_64-linux-gnu/meshlab/plugins 
Opening a file with extension ply
Current Plugins Dir is: /usr/lib/x86_64-linux-gnu/meshlab/plugins 
Opening a file with extension ply
Current Plugins Dir is: /usr/lib/x86_64-linux-gnu/meshlab/plugins 
Opening a file with extension ply


Loading Plugins:
Total 266 filtering actions
Total 13 io plugins
Mesh /home/raman/Desktop/Projects/KineticsVerse/noteboooks/output/mesh31/visual.ply loaded has 1398 vn 2605 fn
output mesh  /home/raman/Desktop/Projects/KineticsVerse/noteboooks/gazebo_model/meshes/mesh31_visual.dae
vertex color, Mesh /home/raman/Desktop/Projects/KineticsVerse/noteboooks/output/mesh31/visual.ply saved as /home/raman/Desktop/Projects/KineticsVerse/noteboooks/gazebo_model/meshes/mesh31_visual.dae (1398 vn 2605 fn)
✓ Saved DAE: gazebo_model/meshes/mesh31_visual.dae
Loading Plugins:
Total 266 filtering actions
Total 13 io plugins
Mesh /home/raman/Desktop/Projects/KineticsVerse/noteboooks/output/mesh31/collision.ply loaded has 814 vn 1499 fn
output mesh  /home/raman/Desktop/Projects/KineticsVerse/noteboooks/gazebo_model/meshes/mesh31_collision.dae
vertex color, Mesh /home/raman/Desktop/Projects/KineticsVerse/noteboooks/output/mesh31/collision.ply saved as /home/raman/Desktop/Projects/KineticsVerse/noteboooks/g

# Test

In [59]:
import os
import json
import numpy as np
import open3d as o3d


class MeshGenerator:
    def __init__(self, out_dir="output"):
        self.out_dir = out_dir
        os.makedirs(out_dir, exist_ok=True)
        self.metadata = {}

    # -----------------------------------------------------
    # Normal estimation
    # -----------------------------------------------------
    def draw_normals(self, pcd):
        pcd = pcd.voxel_down_sample(0.005)
        pcd.estimate_normals(
            search_param=o3d.geometry.KDTreeSearchParamHybrid(
                radius=0.01, max_nn=30
            )
        )
        pcd = pcd.uniform_down_sample(every_k_points=2)
        pcd.orient_normals_consistent_tangent_plane(30)
        return pcd

    # -----------------------------------------------------
    # Poisson reconstruction + density filtering
    # -----------------------------------------------------
    def create_mesh(self, pcd):
        if len(pcd.points) > 1000:
            pcd, _ = pcd.remove_statistical_outlier(30, 2.0)

        pcd = pcd.voxel_down_sample(0.003)

        if len(pcd.points) < 10:
            print("❌ Too few points for meshing.")
            return None

        print("Running Poisson...")

        try:
            mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
                pcd, depth=10
            )
        except:
            print("❌ Poisson crashed.")
            return None

        if len(mesh.vertices) == 0:
            print("❌ Empty Poisson mesh.")
            return None

        densities = np.asarray(densities)
        keep_mask = densities > np.quantile(densities, 0.05)

        verts = np.asarray(mesh.vertices)
        tris = np.asarray(mesh.triangles)

        keep_idx = np.where(keep_mask)[0]
        if len(keep_idx) < 20:
            print("❌ Filtering removed all vertices.")
            return None

        remap = -np.ones(len(verts), dtype=int)
        remap[keep_idx] = np.arange(len(keep_idx))

        good_tris = tris[np.all(keep_mask[tris], axis=1)]
        good_tris = remap[good_tris]

        new_mesh = o3d.geometry.TriangleMesh(
            o3d.utility.Vector3dVector(verts[keep_idx]),
            o3d.utility.Vector3iVector(good_tris.astype(np.int32))
        )

        new_mesh.remove_unreferenced_vertices()
        new_mesh.compute_vertex_normals()

        return new_mesh

    # -----------------------------------------------------
    # Nearest-neighbor color transfer (smooth 5-NN)
    # -----------------------------------------------------
    def color_mesh_from_pcd(self, mesh, original_cluster_pcd):
        if not original_cluster_pcd.has_colors():
            return mesh

        kdtree = o3d.geometry.KDTreeFlann(original_cluster_pcd)
        src_cols = np.asarray(original_cluster_pcd.colors)
        new_cols = []

        for v in mesh.vertices:
            _, idx, dist = kdtree.search_knn_vector_3d(v, 5)
            dist = np.asarray(dist)
            w = 1.0 / (dist + 1e-6)
            w = w / w.sum()
            new_cols.append((src_cols[idx] * w[:, None]).sum(axis=0))

        mesh.vertex_colors = o3d.utility.Vector3dVector(np.asarray(new_cols))
        return mesh

    # -----------------------------------------------------
    def save_metadata(self):
        clean_meta = self._to_python_type(self.metadata)
        with open(os.path.join(self.out_dir, "meta.json"), "w") as f:
            json.dump(clean_meta, f, indent=4)
        print("✓ meta.json saved.")

    # -----------------------------------------------------
    def process_cluster(self, cluster_id, all_points, all_colors, labels, feats):
        idx = np.where(labels == cluster_id)[0]

        if len(idx) < 30:
            print(f"Cluster {cluster_id} too small → skip")
            return

        pts_c = all_points[idx]
        col_c = all_colors[idx]
        feat_c = feats[idx]

        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(pts_c)
        pcd.colors = o3d.utility.Vector3dVector(col_c)

        centroid = pts_c.mean(axis=0).tolist()
        feat_mean = feat_c.mean(axis=0).tolist()

        cluster_dir = os.path.join(self.out_dir, f"mesh{cluster_id}")
        os.makedirs(cluster_dir, exist_ok=True)

        self.metadata[str(cluster_id)] = {
            "cluster_id": cluster_id,
            "centroid": centroid,
            "feature_mean": feat_mean,
            "visual_mesh": f"mesh{cluster_id}/visual.ply",
            "collision_mesh": f"mesh{cluster_id}/collision.ply"
        }

        pcd_norm = self.draw_normals(pcd)
        mesh = self.create_mesh(pcd_norm)
        if mesh is None:
            return

        mesh = self.color_mesh_from_pcd(mesh, pcd)

        vis_path = os.path.join(cluster_dir, "visual.ply")
        o3d.io.write_triangle_mesh(vis_path, mesh, write_vertex_colors=True)

        collision = mesh.simplify_quadric_decimation(1500)
        col_path = os.path.join(cluster_dir, "collision.ply")
        o3d.io.write_triangle_mesh(col_path, collision)

        print(f"✓ Mesh saved for cluster {cluster_id}")

    # -----------------------------------------------------
    def run(self, all_points, all_colors, labels, feats):
        for c in np.unique(labels):
            self.process_cluster(c, all_points, all_colors, labels, feats)
        self.save_metadata()

    def _to_python_type(self, obj):
        if isinstance(obj, dict):
            return {self._to_python_type(k): self._to_python_type(v) for k, v in obj.items()}
        if isinstance(obj, list):
            return [self._to_python_type(i) for i in obj]
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        if isinstance(obj, (np.integer,)):
            return int(obj)
        if isinstance(obj, (np.floating,)):
            return float(obj)
        return obj



In [61]:
mesh_gen = MeshGenerator("output")
mesh_gen.run(pts, color, labels, feats)

Running Poisson...
✓ Mesh saved for cluster -1
Running Poisson...
✓ Mesh saved for cluster 0
Running Poisson...
✓ Mesh saved for cluster 1
Running Poisson...
✓ Mesh saved for cluster 2
Running Poisson...
✓ Mesh saved for cluster 3
Cluster 4 too small → skip
Running Poisson...
✓ Mesh saved for cluster 5
Running Poisson...
✓ Mesh saved for cluster 6
Running Poisson...
✓ Mesh saved for cluster 7
Cluster 8 too small → skip
Running Poisson...
✓ Mesh saved for cluster 9
Running Poisson...
✓ Mesh saved for cluster 10
Running Poisson...
✓ Mesh saved for cluster 11
Running Poisson...
✓ Mesh saved for cluster 12
Running Poisson...
✓ Mesh saved for cluster 13
Running Poisson...
✓ Mesh saved for cluster 14
Running Poisson...
✓ Mesh saved for cluster 15
Running Poisson...
✓ Mesh saved for cluster 16
Cluster 17 too small → skip
Running Poisson...
✓ Mesh saved for cluster 18
Running Poisson...
✓ Mesh saved for cluster 19
Running Poisson...
✓ Mesh saved for cluster 20
Running Poisson...
✓ Mesh saved f

In [62]:
import os
import json
import numpy as np
import open3d as o3d
import subprocess


class GazeboModelBuilder:
    def __init__(self,
                 output_root="output",
                 gazebo_root="gazebo_model",
                 model_name="unified_scene"):

        self.output_root = output_root
        self.gazebo_root = gazebo_root
        self.model_name = model_name

        self.mesh_dir = os.path.join(gazebo_root, "meshes")
        os.makedirs(self.mesh_dir, exist_ok=True)

        self.valid_clusters = {}

    # --------------------------------------------------------------
    # PLY → OBJ (keeps vertex colors + normals + auto MTL)
    # --------------------------------------------------------------
    def convert_ply_to_obj(self, ply_path, obj_path):
        if not os.path.exists(ply_path):
            print(f"❌ Missing file: {ply_path}")
            return False

        print(f"→ Converting {ply_path} → {obj_path}")

        mesh = o3d.io.read_triangle_mesh(ply_path)
        if mesh.is_empty():
            print("❌ Mesh empty")
            return False

        o3d.io.write_triangle_mesh(
            obj_path,
            mesh,
            write_vertex_colors=True,
            write_ascii=False
        )

        # ensure .mtl exists
        mtl_path = obj_path.replace(".obj", ".mtl")
        if os.path.exists(obj_path) and os.path.exists(mtl_path):
            print(f"✓ Saved OBJ + MTL: {obj_path}")
            return True
        else:
            print(f"⚠ OBJ OK but MTL missing: {mtl_path}")
            return True

    # --------------------------------------------------------------
    # PLY → DAE (collision only)
    # --------------------------------------------------------------
    def convert_ply_to_dae(self, ply_path, dae_path):
        cmd = [
            "meshlabserver",
            "-i", ply_path,
            "-o", dae_path,
            "-m", "vn"
        ]
        try:
            subprocess.run(cmd, check=True)
            print(f"✓ Saved DAE: {dae_path}")
            return True
        except:
            print(f"❌ DAE failed: {ply_path}")
            return False

    # --------------------------------------------------------------
    def write_model_config(self):
        cfg = os.path.join(self.gazebo_root, "model.config")
        with open(cfg, "w") as f:
            f.write(f"""<?xml version="1.0"?>
<model>
  <name>{self.model_name}</name>
  <version>1.0</version>
  <sdf version="1.6">model.sdf</sdf>
</model>""")
        print(f"✓ Wrote {cfg}")

    # --------------------------------------------------------------
    def write_model_sdf(self):
        sdf = os.path.join(self.gazebo_root, "model.sdf")

        with open(sdf, "w") as f:
            f.write(f"""<?xml version="1.0"?>
<sdf version="1.6">
  <model name="{self.model_name}">
    <static>true</static>
""")

        with open(sdf, "a") as f:
            for cid, entry in self.valid_clusters.items():
                cx, cy, cz = entry["centroid"]
                vis_obj = f"meshes/mesh{cid}_visual.obj"
                col_dae = f"meshes/mesh{cid}_collision.dae"

                f.write(f"""
    <link name="cluster_{cid}">
      <pose>{cx} {cy} {cz} 0 0 0</pose>

      <visual name="visual_{cid}">
        <geometry>
          <mesh>
            <uri>model://{self.model_name}/{vis_obj}</uri>
            <scale>1 1 1</scale>
          </mesh>
        </geometry>
      </visual>

      <collision name="collision_{cid}">
        <geometry>
          <mesh>
            <uri>model://{self.model_name}/{col_dae}</uri>
            <scale>1 1 1</scale>
          </mesh>
        </geometry>
      </collision>
    </link>
""")

        with open(sdf, "a") as f:
            f.write("""
  </model>
</sdf>
""")

        print(f"✓ Wrote {sdf}")

    # --------------------------------------------------------------
    def build(self):
        meta_file = os.path.join(self.output_root, "meta.json")

        if not os.path.exists(meta_file):
            print("❌ meta.json missing")
            return

        meta = json.load(open(meta_file))

        print("✓ Loaded metadata")

        for cid, entry in meta.items():
            cid = int(cid)
            mesh_dir = os.path.join(self.output_root, f"mesh{cid}")

            vis_ply = os.path.join(mesh_dir, "visual.ply")
            col_ply = os.path.join(mesh_dir, "collision.ply")

            vis_obj = os.path.join(self.mesh_dir, f"mesh{cid}_visual.obj")
            col_dae = os.path.join(self.mesh_dir, f"mesh{cid}_collision.dae")

            ok1 = self.convert_ply_to_obj(vis_ply, vis_obj)
            ok2 = self.convert_ply_to_dae(col_ply, col_dae)

            if ok1 and ok2:
                self.valid_clusters[cid] = entry

        self.write_model_config()
        self.write_model_sdf()

        print("\n🎉 DONE — OBJ visuals + DAE collisions included:", len(self.valid_clusters))
        print(f"👉 Copy to ~/.gazebo/models/{self.model_name} and spawn it.")

In [63]:
builder = GazeboModelBuilder(output_root="output",
                             gazebo_root="gazebo_model",
                             model_name="unified_scene")
builder.build()

✓ Loaded metadata
→ Converting output/mesh-1/visual.ply → gazebo_model/meshes/mesh-1_visual.obj
⚠ OBJ OK but MTL missing: gazebo_model/meshes/mesh-1_visual.mtl


Current Plugins Dir is: /usr/lib/x86_64-linux-gnu/meshlab/plugins 
Opening a file with extension ply


Loading Plugins:
Total 266 filtering actions
Total 13 io plugins
Mesh /home/raman/Desktop/Projects/KineticsVerse/noteboooks/output/mesh-1/collision.ply loaded has 982 vn 1499 fn
output mesh  /home/raman/Desktop/Projects/KineticsVerse/noteboooks/gazebo_model/meshes/mesh-1_collision.dae
vertex normals, Mesh /home/raman/Desktop/Projects/KineticsVerse/noteboooks/output/mesh-1/collision.ply saved as /home/raman/Desktop/Projects/KineticsVerse/noteboooks/gazebo_model/meshes/mesh-1_collision.dae (982 vn 1499 fn)
✓ Saved DAE: gazebo_model/meshes/mesh-1_collision.dae
→ Converting output/mesh0/visual.ply → gazebo_model/meshes/mesh0_visual.obj
⚠ OBJ OK but MTL missing: gazebo_model/meshes/mesh0_visual.mtl
Loading Plugins:
Total 266 filtering actions
Total 13 io plugins
Mesh /home/raman/Desktop/Projects/KineticsVerse/noteboooks/output/mesh0/collision.ply loaded has 3019 vn 2241 fn
output mesh  /home/raman/Desktop/Projects/KineticsVerse/noteboooks/gazebo_model/meshes/mesh0_collision.dae
vertex norma

Current Plugins Dir is: /usr/lib/x86_64-linux-gnu/meshlab/plugins 
Opening a file with extension ply
Current Plugins Dir is: /usr/lib/x86_64-linux-gnu/meshlab/plugins 


⚠ OBJ OK but MTL missing: gazebo_model/meshes/mesh1_visual.mtl
Loading Plugins:
Total 266 filtering actions
Total 13 io plugins
Mesh /home/raman/Desktop/Projects/KineticsVerse/noteboooks/output/mesh1/collision.ply loaded has 786 vn 1499 fn
output mesh  /home/raman/Desktop/Projects/KineticsVerse/noteboooks/gazebo_model/meshes/mesh1_collision.dae
vertex normals, Mesh /home/raman/Desktop/Projects/KineticsVerse/noteboooks/output/mesh1/collision.ply saved as /home/raman/Desktop/Projects/KineticsVerse/noteboooks/gazebo_model/meshes/mesh1_collision.dae (786 vn 1499 fn)
✓ Saved DAE: gazebo_model/meshes/mesh1_collision.dae
→ Converting output/mesh2/visual.ply → gazebo_model/meshes/mesh2_visual.obj
⚠ OBJ OK but MTL missing: gazebo_model/meshes/mesh2_visual.mtl


...

Opening a file with extension ply
Current Plugins Dir is: /usr/lib/x86_64-linux-gnu/meshlab/plugins 
Opening a file with extension ply


Loading Plugins:
Total 266 filtering actions
Total 13 io plugins
Mesh /home/raman/Desktop/Projects/KineticsVerse/noteboooks/output/mesh25/collision.ply loaded has 813 vn 1500 fn
output mesh  /home/raman/Desktop/Projects/KineticsVerse/noteboooks/gazebo_model/meshes/mesh25_collision.dae
vertex normals, Mesh /home/raman/Desktop/Projects/KineticsVerse/noteboooks/output/mesh25/collision.ply saved as /home/raman/Desktop/Projects/KineticsVerse/noteboooks/gazebo_model/meshes/mesh25_collision.dae (813 vn 1500 fn)
✓ Saved DAE: gazebo_model/meshes/mesh25_collision.dae
→ Converting output/mesh29/visual.ply → gazebo_model/meshes/mesh29_visual.obj
⚠ OBJ OK but MTL missing: gazebo_model/meshes/mesh29_visual.mtl
Loading Plugins:
Total 266 filtering actions
Total 13 io plugins
Mesh /home/raman/Desktop/Projects/KineticsVerse/noteboooks/output/mesh29/collision.ply loaded has 818 vn 1500 fn
output mesh  /home/raman/Desktop/Projects/KineticsVerse/noteboooks/gazebo_model/meshes/mesh29_collision.dae
vertex n

Current Plugins Dir is: /usr/lib/x86_64-linux-gnu/meshlab/plugins 
Opening a file with extension ply
Current Plugins Dir is: /usr/lib/x86_64-linux-gnu/meshlab/plugins 


⚠ OBJ OK but MTL missing: gazebo_model/meshes/mesh31_visual.mtl
Loading Plugins:
Total 266 filtering actions
Total 13 io plugins
Mesh /home/raman/Desktop/Projects/KineticsVerse/noteboooks/output/mesh31/collision.ply loaded has 814 vn 1499 fn
output mesh  /home/raman/Desktop/Projects/KineticsVerse/noteboooks/gazebo_model/meshes/mesh31_collision.dae
vertex normals, Mesh /home/raman/Desktop/Projects/KineticsVerse/noteboooks/output/mesh31/collision.ply saved as /home/raman/Desktop/Projects/KineticsVerse/noteboooks/gazebo_model/meshes/mesh31_collision.dae (814 vn 1499 fn)
✓ Saved DAE: gazebo_model/meshes/mesh31_collision.dae
✓ Wrote gazebo_model/model.config
✓ Wrote gazebo_model/model.sdf

🎉 DONE — OBJ visuals + DAE collisions included: 25
👉 Copy to ~/.gazebo/models/unified_scene and spawn it.


Opening a file with extension ply
